In [64]:
import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 3
tau = torch.tensor(100, device=device)
evaporation_rate = torch.exp(-1 / tau)
beta = torch.tensor(0.001, device=device)
start_seed = 43
start_alpha = 0.1
iter = 4
energy_series = []
m_series = []

h, J, st1, st0, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):

    prob = []


    determined_spins = []
    determined_spins.append(2 * (torch.rand(1) > 0.5) - 1)
    for new_spins in range(1, n):
        external_effect = simulation.calculate_external_effect(determined_spins, J)
        new_spin = 2 * (simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect) > torch.rand(1)) - 1
        prob.append(simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect))
        determined_spins.append(new_spin)
    determined_spins = torch.tensor(determined_spins)
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((determined_spins + 1) / 2)
    st0 = st0 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((-determined_spins + 1) / 2)
    print(determined_spins)
    print(prob)
    print(st1)
    print(st0)
    

#determined_spins.append((2 * (0.5 > torch.rand(1)) - 1))
#determined_spins.append(torch.tensor([-1]))
#determined_spins = torch.tensor(determined_spins)
#print(determined_spins)
#print(J)
#print(calculate(determined_spins, J))

tensor([ 1,  1, -1])
[tensor([0.5500]), tensor([0.4500])]
tensor([2.9643, 2.9643, 0.0000])
tensor([0.0000, 0.0000, 2.9643])
tensor([ 1,  1, -1])
[tensor([0.5500]), tensor([0.4500])]
tensor([5.5155, 5.5155, 0.0000])
tensor([0.0000, 0.0000, 5.5155])
tensor([-1, -1,  1])
[tensor([0.5500]), tensor([0.4500])]
tensor([5.4606, 5.4606, 2.5755])
tensor([2.5755, 2.5755, 5.4606])
tensor([ 1, -1, -1])
[tensor([0.5180]), tensor([0.4820])]
tensor([6.2932, 5.4063, 2.5499])
tensor([2.5499, 3.4368, 6.2932])


In [98]:
import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
tau = torch.tensor(100, device=device)
evaporation_rate = torch.exp(-1 / tau)
beta = torch.tensor(0.001, device=device)
start_seed = 42
start_alpha = 0.0
iter = 1e5
energy_series = []
m_series = []
z_series = []

h, J, st1, st0, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):

    prob = []

    determined_spins = []
    determined_spins.append(2 * (torch.rand(1) > 0.5) - 1)
    for new_spins in range(1, n):
        external_effect = simulation.calculate_external_effect(determined_spins, J)
        new_spin = 2 * (simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect) > torch.rand(1)) - 1
        prob.append(simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect))
        determined_spins.append(new_spin)
    determined_spins = torch.tensor(determined_spins)
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((determined_spins + 1) / 2)
    st0 = st0 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((-determined_spins + 1) / 2)
    z = st1 / (st1 + st0)
    alpha += alpha_inc
    m = 2 * alpha * (z - 0.5)
    energy_series.append(simulation.calculate_energy(determined_spins, h, J).item())
    m_series.append(m.tolist())
    #z_series.append(z.tolist())    

energy_series.insert(0, 'energy')
energy_file_name = f'tau{tau:.0e}_beta{beta:.0e}_energy'
m_file_name = f'tau{tau:.0e}_beta{beta:.0e}_m'
#z_file_name = f'tau{tau:.0e}_beta{beta:.0e}_z'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)
#output.matrix_to_csv(z_file_name, z_series, start_seed, iter)

In [94]:
#non annealing

import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
tau = torch.tensor(100, device=device)
evaporation_rate = torch.exp(-1 / tau)
beta = torch.tensor(0.01, device=device)
start_seed = 42
start_alpha = 0.0
iter = 1e4
energy_series = []
m_series = []
z_series = []

h, J, st1, st0, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

alpha = torch.tensor(0.5, device=device)

for i in range(int(iter)):

    prob = []

    determined_spins = []
    determined_spins.append(2 * (torch.rand(1) > 0.5) - 1)
    for new_spins in range(1, n):
        external_effect = simulation.calculate_external_effect(determined_spins, J)
        new_spin = 2 * (simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect) > torch.rand(1)) - 1
        prob.append(simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect))
        determined_spins.append(new_spin)
    determined_spins = torch.tensor(determined_spins)
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((determined_spins + 1) / 2)
    st0 = st0 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((-determined_spins + 1) / 2)
    z = st1 / (st1 + st0)
    energy_series.append(simulation.calculate_energy(determined_spins, h, J).item())
    z_series.append(z.tolist())    

energy_series.insert(0, 'energy')
energy_file_name = f'nonannealing_tau{tau:.0e}_beta{beta:.0e}_energy'
z_file_name = f'nonannealing_tau{tau:.0e}_beta{beta:.0e}_z'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(z_file_name, z_series, start_seed, iter)

In [77]:
import torch
import simulation
import output
import importlib
importlib.reload(simulation)
importlib.reload(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 3
tau = torch.tensor(100, device=device)
evaporation_rate = torch.exp(-1 / tau)
beta = torch.tensor(0.001, device=device)
start_seed = 42
start_alpha = 0.0
iter = 4
energy_series = []
m_series = []
z_series = []

h, J, st1, st0, alpha, alpha_inc = simulation.initialize_spinglass(n, start_seed, iter, start_alpha, device)

for i in range(int(iter)):

    prob = []

    determined_spins = []
    determined_spins.append(2 * (torch.rand(1) > 0.5) - 1)
    for new_spins in range(1, n):
        external_effect = simulation.calculate_external_effect(determined_spins, J)
        new_spin = 2 * (simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect) > torch.rand(1)) - 1
        prob.append(simulation.decision_probabilities(st1, st0, len(determined_spins), alpha, beta, external_effect))
        determined_spins.append(new_spin)
    determined_spins = torch.tensor(determined_spins)
    st1 = st1 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((determined_spins + 1) / 2)
    st0 = st0 * evaporation_rate + torch.exp(-simulation.calculate_energy(determined_spins, h, J)) * ((-determined_spins + 1) / 2)
    z = st1 / (st1 + st0)
    alpha += alpha_inc
    m = 2 * alpha * (z - 0.5)
    if i % int(iter / 1e6) == 0:
        energy_series.append(simulation.calculate_energy(determined_spins, h, J).item())
        m_series.append(m.tolist())
        z_series.append(z.tolist())    

energy_series.insert(0, 'energy')
energy_file_name = f'tau{tau:.0e}_beta{beta:.0e}_energy'
m_file_name = f'tau{tau:.0e}_beta{beta:.0e}_m'
z_file_name = f'tau{tau:.0e}_beta{beta:.0e}_z'
output.list_to_csv(energy_file_name, energy_series, start_seed, iter)
output.matrix_to_csv(m_file_name, m_series, start_seed, iter)
output.matrix_to_csv(z_file_name, z_series, start_seed, iter)

ZeroDivisionError: integer modulo by zero